In [7]:
## for data
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.axes
## for statistical tests
import scipy
# import statsmodels.formula.api as smf
# import statsmodels.api as sm
## for machine learning
from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition
## for explainer
# from lime import lime_tabular
import matplotlib.ticker as plticker
import pickle

In [8]:
#Predit on the test set
dfv = pd.read_csv("test.csv")

In [9]:
dfv = pd.get_dummies(dfv, columns=['Marital status'])
dfv = pd.get_dummies(dfv, columns=['Application mode'])
dfv = pd.get_dummies(dfv, columns=['Course'])
dfv = pd.get_dummies(dfv, columns=['Previous qualification'])
dfv = pd.get_dummies(dfv, columns=['Nacionality'])
dfv = pd.get_dummies(dfv, columns=["Mother's occupation"])
dfv = pd.get_dummies(dfv, columns=["Father's occupation"])

#DROP AND CREATE
dfv = dfv.drop(["Mother's qualification", "Father's qualification"], axis=1)
dfv["valid_enroll_sem2"] = dfv["Curricular units 2nd sem (enrolled)"] - dfv["Curricular units 2nd sem (approved)"]
dfv["valid_enroll_sem1"] = dfv["Curricular units 1st sem (enrolled)"] - dfv["Curricular units 1st sem (approved)"]
dfv["gap_enroll_valid"] = dfv["valid_enroll_sem1"] - dfv["valid_enroll_sem2"]

#MinMax-Scaler
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(dfv.drop(["Id"], axis=1))
dtf_scaled= pd.DataFrame(X, columns=dfv.drop(["Id"], axis=1).columns)
dtf_scaled["Id"] = dfv["Id"]
dfv = dtf_scaled

#features selection
dfv = dfv[["Id", "valid_enroll_sem2", 
"gap_enroll_valid",
"Curricular units 2nd sem (grade)",
"Age at enrollment",
"Tuition fees up to date",
"Scholarship holder"]]


#Validation score see

X_test = dfv[["valid_enroll_sem2", 
"gap_enroll_valid",
"Curricular units 2nd sem (grade)",
"Age at enrollment",
"Tuition fees up to date",
"Scholarship holder"]].values
loaded_model = pickle.load(open("finalized_model_academic_V5.sav", 'rb'))
result = loaded_model.predict(X_test)
submit_df = pd.DataFrame()
submit_df['Id'] = dfv["Id"]
submit_df['Category'] = result
submit_df.to_csv("sample_submission_5bis.csv", index= False)